## Clustering 

The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [1]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

Import libraries

In [2]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. 

In [3]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [7]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Clustering

In [8]:
#Get a feature collection with Egypt
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Egypt = countries.filter(ee.Filter.eq('country_na', 'Egypt'))

# Load a pre-computed Landsat composite for input.
input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

# Define a region in which to generate a sample of the input.
region = ee.Geometry.Rectangle(-123.25, 48.29, -123.95, 48.72)

# Display the sample region.
Map.setCenter(31.5, 31.0, 8)
Map.addLayer(ee.Image().paint(region, 0, 2), {}, 'region')

# Make the training dataset.
training = input.sample(**{
  'region': region,
  'scale': 30,
  'numPixels': 5000
})

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = input.cluster(clusterer)

# Display the clusters with random colors.
Map.addLayer(result.clip(Egypt).randomVisualizer(), {}, 'clusters')


## Display

In [9]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map